# Sample Template

Here is some sample text, and a first block of code

In [ ]:
import re

text = r"""
This is a text with some math blocks: $E=mc^2$, $g(x)=\int_{-\infty}^x f(t) dt$ and
$ f(x) = \{ x: \text{if $x<0$} ] $, but we also have false positives like $ cd $"""

regex = r"""
(?<!\$)     # Negative lookbehind assertion to exclude $
(?P<dollar>\$)   # Match and capture the first $
(?!\s)      # Negative lookahead assertion to exclude whitespace
(?P<content>
  (?:                   # Non-capturing group
    [^{${]+             # Match any character except {, $ and newline
    |                   # OR
    \{                  # Match the opening curly brace
      (?:[^{}]+|        # Match any character except { and } OR
        \{(?:[^{}]+|\{(?1)\})*\}  # Match balanced braces recursively
      )*                # Match zero or more times
      [^{}]*            # Match any character except { and } zero or more times
    \}                  # Match the closing curly brace
  )+
)
\s*         # Match any whitespace characters
(?<!\s)     # Negative lookbehind assertion to exclude whitespace
(?P<right>\$)    # Match and capture the second $
(?!        # Negative lookahead assertion to exclude $
  (?<=})   # Match only if the last matching group is not a closing curly brace
)
"""

text = r"The equation $E=mc^2$ is famous. But what about $$F=ma$$ or $g(x)=\int_{-\infty}^x f(t) dt$? Or $ f(x) = \{ x: \text{if $x<0$} ] $?"

matches = re.findall(regex, text, re.MULTILINE | re.DOTALL | re.VERBOSE)

print(matches)

In [ ]:
import re

regex = r"""
    (?<!\$)        # Negative lookbehind for '$'
    (?P<dollar>\$) # Match the opening dollar sign and save it as a group named 'dollar'
    (?!\s)         # Negative lookahead for whitespace
    (?P<content>   # Match the content between the dollar signs and save it as a group named 'content'
        (?:        # Non-capturing group for the content options
            [^{$]+ # Match any character that is not '{' or '$' one or more times
            |      # or
            \{     # Match the opening curly brace
                (?:          # Non-capturing group for nested curly braces
                    [^{}]+   # Match any character that is not '{' or '}' one or more times
                    |        # or
                    \{[^{}]*\} # Match nested curly braces
                )*           # Match the non-capturing group zero or more times
            \}     # Match the closing curly brace
        )+          # Require the content to have at least one character
    )              
    \s*            # Match any whitespace characters
    (?<!\s)        # Negative lookbehind for whitespace
    (?P<right>\$)  # Match the closing dollar sign and save it as a group named 'right'
    (?!\$)         # Negative lookahead for '$'
"""

# Compile the regex with the VERBOSE flag
pattern = re.compile(regex, re.VERBOSE)

# Test the regex
text = r"$ f(x) = \{ x: \text{if $x<0$} ] $   The equation $E=mc^2$ is famous. But what about $$F=ma$$ or $g(x)=\int_{-\infty}^x f(t) dt$?"
matches = pattern.findall(text)
print(matches)

In [ ]:
regex = r"""
(?<!\$)              # Negative lookbehind to ensure that there is no $ before the first $
(?P<left>\$)         # Capture the first $ in a named group "dollar"
(?!\s)               # Negative lookahead to ensure that there is no whitespace after the first $
(?P<content>         # Capture the content of the math block in a named group "content"
  (?:                # Non-capturing group for the content
    [^{$]+           # Match any character that is not { or $
    |                # OR
    (?:(?<!\\)\{     # Match an opening curly brace
      (?:            # Non-capturing group for the nested content within curly braces
        [^{}]+       # Match any character that is not { or }
        |            # OR
        \{    # Match an opening curly brace
          [^{}]*     # Match any characters that are not { or } zero or more times
        \}    # Match a closing curly brace
      )*             # Match the nested content group zero or more times
    (?<!\\)\})       # Match a closing curly brace
  )+                 # Match the non-capturing group one or more times
)                    #
(?<!\s)              # Negative lookbehind to ensure that there is no whitespace before the final $
(?P<right>\$)        # Capture the final $ in a named group "right"
(?!\$)               # Negative lookahead to ensure that there is no $ after the final $
"""

In [ ]:
## MY SOLUTION

import re

regex = r"""
(?<!\$)                   # Negative lookbehind to ensure that there is no $ before the first $
(?<!\\)                   # Negative lookbehind to ensure there is no \ before the first $
(?P<left>\$)              # Capture the first $ in a named group "dollar"
(?!\$)                    # Negative lookahead to ensure that there is no $ after the first $
(?!\s)                    # Negative lookahead to ensure that there is no whitespace after the first $
(?P<content>              # Capture the content of the math block in a named group "content"
  (?:                     # Non-capturing group for the content
    ([^{$]|\\\{|\\\$|)+   # Match any character that is not { or $, excluding '\{' and '\$'
    |                     # OR
    \{                    # Match opening brace
    ([^{]|)               # Match any characters, excluding curly brace, exclusing '\{'
    \}                    # Match a closing curly brace

    (
    (?<!\\)\{        # Match an opening curly brace, but not \{
      (?:            # Non-capturing group for the nested content within curly braces
        [^{}]+       # Match any character that is not { or }
        |            # OR
        \{           # Match an opening curly brace
          [^{}]*     # Match any characters that are not { or } zero or more times
        \}           # Match a closing curly brace
      )*             # Match the nested content group zero or more times
    (?<!\\)\}        # Match a closing curly brace, but not \}
    )
  )+                 # Match the non-capturing group one or more times
)                    #
(?<!\$)              # Negative lookbehind to ensure that there is no $ before the final $
(?<!\\)              # Negative lookbehind to ensure there is no \ before the final $
(?<!\s)              # Negative lookbehind to ensure that there is no whitespace before the final $
(?P<right>\$)        # Capture the final $ in a named group "right"
(?!\$)               # Negative lookahead to ensure that there is no $ after the final $
"""


# regex = r"(?<!\$)(?P<left>\$)(?!\s)(?P<content>(?:[^{$]+|\{(?:[^{}]+|\{[^{}]*\})*\})+)\s*(?<!\s)(?P<right>\$)(?!\$)"


# Compile the regex with the VERBOSE flag
pattern = re.compile(regex, re.VERBOSE)

# Test the regex

text = r"""
The equation $E=mc^2$ is famous. But what about $$F=ma$$ or $g(x)=\int_{-\infty}^x f(t) dt$?

Here is some text with a math block: $x = \frac{1}{2}$.

Here is another math block with nested math:
$f(x) = y \text{ for $x \in \mathbb{R}$ and $y \in \{1,2,3\}$}$

Here is a code block: `$ cd ..`.

multi-line example: $f(x)=
y$

$f(x) = \{ x: \text{if $x<0$} ]$
"""

matches = pattern.findall(text)
print(matches)

In [ ]:
import re

regex = r"""
    (?<!\$)                    # Negative lookbehind for '$'
    (?P<dollar>\$)             # Match the opening dollar sign and save it as a group named 'dollar'
    (?!\s)                     # Negative lookahead for whitespace
    (?P<content>               # Match the content between the dollar signs and save it as a group named 'content'
        (?:                    # Non-capturing group for the content options
            [^{$]+             # Match any character that is not '{' or '$' one or more times
            |                  # or
            (?<!\\)\{          # Match the opening curly brace
                (?:            # Non-capturing group for nested curly braces
                    [^{}]+     # Match any character that is not '{' or '}' one or more times
                    |          # or
                    \{[^{}]*\} # Match nested curly braces
                )*             # Match the non-capturing group zero or more times
            (?<!\\)\}          # Match the closing curly brace
        )+                     # Require the content to have at least one character
    )                          #
    \s*                        # Match any whitespace characters
    (?<!\s)                    # Negative lookbehind for whitespace
    (?P<right>\$)              # Match the closing dollar sign and save it as a group named 'right'
    (?!\$)                     # Negative lookahead for '$'
"""

# Compile the regex with the VERBOSE flag
pattern = re.compile(regex, re.VERBOSE)

# Test the regex
# text = "The equation $E=mc^2$ is famous. But what about $$F=ma$$ or $g(x)=\int_{-\infty}^x f(t) dt$?"
matches = pattern.findall(text)
print(matches)

In [ ]:
import re

text = """
Here is some text with a math block: $x = \frac{1}{2}$.

Here is another math block with nested math:
$f(x) = y \text{ for $x \in \mathbb{R}$ and $y \in \{1,2,3\}$}$

Here is a code block: `$ cd ..`.
"""

MY_REGEX = (
    r"(?<!\$)(?P<left>\$)(?!\s)"  # match the leading dollar, but no double dollar.
    r"(?<!\s)(?P<right>\$)(?!\$)"  # match the end dollar, but no double dollar.
)


regex = (
    r"(?<!\$)(?P<dollar>\$)(?!\s)"
    r"(?P<content>(?:[^{$]+|\{(?:[^{}]+|\{[^{}]*\})*\})*)\s*"
    r"(?<!\s)(?P=dollar)(?!\$)"
)

alt = (
    r"(?<!\$)(?P<left>\$)"
    r"(?!\s)(?P<content>(?:[^{$]+|\{(?:[^{}]+|\{[^{}]*\})*\})*)\s*"
    r"(?<!\s)(?P<right>\$)(?!\$)"
)


display

pattern = re.compile(
    regex,
    re.DOTALL | re.MULTILINE,
)
matches = pattern.findall(text)

for match in matches:
    display(f"{match[1]:s}")  # Print the inner content of each math block

## A second heading

and some more text

In [ ]:
import jax.numpy as jnp
import numpy as np
from jax import vmap

x = np.array([3, 4, 2])
y = np.array([5, 2, 3, 6])

print(np.multiply.outer(x, y))
# [[15  6  9 18]
#  [20  8 12 24]
#  [10  4  6 12]]
print(vmap(vmap(jnp.multiply, (None, 0)), (0, None))(x, y))
# [[15  6  9 18]
#  [20  8 12 24]
#  [10  4  6 12]]

In [ ]:
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
np.add.outer(x, y)

In [ ]:
vmap(vmap(jnp.add, (None, None, 0)), (0, None, None))(x, y)